<a href="https://colab.research.google.com/github/MWFK/NLP-Semantic-Similarity/blob/main/ClinicalTrials/Data%20Engineering/03.%20InclusionCriteria%20N-Grams%20Frequencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import re
import pandas as pd
# pd.set_option('display.max_columns', None)  
# pd.set_option('display.max_colwidth', None)
import requests
from itertools import compress
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet

from google.colab import files

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
####### Search Expression #######
# None

####### Study Fields #######
# NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria 	

####### New Fields to add #######
# LocationStatus, Phase

####### Range Min_MAX ######
# 1 to 1000

####### Format #######
# CSV

url = 'https://clinicaltrials.gov/api/query/study_fields?expr=&fields=NCTId%2CBriefTitle%2CCondition%2CKeyword%2CEligibilityCriteria+%09&min_rnk=1&max_rnk=1000&fmt=csv'
session = requests.Session()
retry   = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://' , adapter)
session.mount('https://', adapter)

clinicaltrials = session.get(url)
print('Download Request Status: ', clinicaltrials.status_code)

csv_file = open('/content/'+str(1)+'-batch.csv', 'wb')
csv_file.write(clinicaltrials.content)
csv_file.close()

df = pd.read_csv(r'/content/1-batch.csv', skiprows=10)
print(df.shape)
df.head()

Download Request Status:  200
(1000, 6)


,Rank,NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria
0,1,NCT05237167,Time to Diagnosis of Glenohumeral Joint Disloc...,Glenohumeral Dislocation,glenohumeral dislocation|POC Ultrasound|Plain ...,Inclusion Criteria:||Adult patients (19 and ol...
1,2,NCT05237154,Intermittent Fasting and Exercise: Effects on ...,Composition and Function of the Gut Microbiota...,Obesity|Gut microbiota|Intermittent fasting|Hi...,Inclusion criteria:||Age between 18 and 40 yea...
2,3,NCT05237141,Etonogestrel Implant as Emergency Contraception,Contraception,NaN,Inclusion Criteria:||BMI less than or equal to...
3,4,NCT05237128,The Lifestyle Education About Nutrition for Di...,Type 2 Diabetes,Very low-carbohydrate diet|Moderate-carbohydra...,Inclusion Criteria:||diagnosis of type 2 diabe...
4,5,NCT05237115,Helicobacter Pylori Eradication With Probiotic...,Gastritis|Peptic Ulcer|Dyspepsia,Helicobacter pylori|Probiotics-combining tripl...,"Inclusion Criteria:||Age between 18~75,both ge..."


In [3]:
# Convert the EligibilityCriteria to string then lowercase
EligibilityCriteria = df['EligibilityCriteria'].astype(str).str.lower().tolist()
EligibilityCriteria[:10]

['inclusion criteria:||adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation||exclusion criteria:||injury sustained in major traumatic event (trauma activation), unable to consent, in extremis, less then 19 years of age',
 'inclusion criteria:||age between 18 and 40 years;|women;|have a regular menstrual cycle;|body weight below 120 kg;|bmi between 30 kg/m2 and 40 kg/m2.||exclusion criteria:||individuals with any disease;|smokers;|alcoholics;|use regular medication;|medical impediment to the practice of physical activities;|carry out nutritional monitoring;|being in treatment for weight loss;|present some discomfort in long periods of fasting.',
 'inclusion criteria:||bmi less than or equal to 28kg/m2|intact uterus with at least one ovary|regular menstrual cycles that occur every 21-35 days|if patient is postpartum or post-second trimester abortion, 3 menses (2 cycles) must have occurred p

### N-Grams Frequencies

In [4]:
text = 'this is a foo bar sentences and this is my objective : to ngramize it'
ngram = 2 # 15 is the longest sequence, 16 gives an empty list, 17 raises an iteration error
print(word_tokenize(text))
#nltk.FreqDist(text.split())

['this', 'is', 'a', 'foo', 'bar', 'sentences', 'and', 'this', 'is', 'my', 'objective', ':', 'to', 'ngramize', 'it']


In [5]:
def ngrams_split(text, ngram):
    n_grams = ngrams(word_tokenize(text), ngram)
    return [ ' '.join(grams) for grams in n_grams]

ngrams_split(text, ngram)

['this is',
 'is a',
 'a foo',
 'foo bar',
 'bar sentences',
 'sentences and',
 'and this',
 'this is',
 'is my',
 'my objective',
 'objective :',
 ': to',
 'to ngramize',
 'ngramize it']

In [6]:
def ngrams_frequency(text, ngram):
  bigram_fd = nltk.FreqDist(ngrams_split(text, ngram))
  return bigram_fd.most_common()

ngrams_frequency(text, ngram)  

[('this is', 2),
 ('is a', 1),
 ('a foo', 1),
 ('foo bar', 1),
 ('bar sentences', 1),
 ('sentences and', 1),
 ('and this', 1),
 ('is my', 1),
 ('my objective', 1),
 ('objective :', 1),
 (': to', 1),
 ('to ngramize', 1),
 ('ngramize it', 1)]

### InclusionCriteria Seperation

In [7]:
# Convert the EligibilityCriteria to string then lowercase
EligibilityCriteria = df['EligibilityCriteria'].astype(str).str.lower().tolist()
EligibilityCriteria[:5]

['inclusion criteria:||adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation||exclusion criteria:||injury sustained in major traumatic event (trauma activation), unable to consent, in extremis, less then 19 years of age',
 'inclusion criteria:||age between 18 and 40 years;|women;|have a regular menstrual cycle;|body weight below 120 kg;|bmi between 30 kg/m2 and 40 kg/m2.||exclusion criteria:||individuals with any disease;|smokers;|alcoholics;|use regular medication;|medical impediment to the practice of physical activities;|carry out nutritional monitoring;|being in treatment for weight loss;|present some discomfort in long periods of fasting.',
 'inclusion criteria:||bmi less than or equal to 28kg/m2|intact uterus with at least one ovary|regular menstrual cycles that occur every 21-35 days|if patient is postpartum or post-second trimester abortion, 3 menses (2 cycles) must have occurred p

In [8]:
# Seperate the InclusionCriteria from the ExclusionCriteria
InclusionCriteria = [txt[21:txt.find('exclusion criteria')-2] for txt in EligibilityCriteria]
InclusionCriteria[:5]

['adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation',
 'age between 18 and 40 years;|women;|have a regular menstrual cycle;|body weight below 120 kg;|bmi between 30 kg/m2 and 40 kg/m2.',
 'bmi less than or equal to 28kg/m2|intact uterus with at least one ovary|regular menstrual cycles that occur every 21-35 days|if patient is postpartum or post-second trimester abortion, 3 menses (2 cycles) must have occurred prior to enrollment|if patient had a first trimester abortion or pregnancy loss, she must have one spontaneous menses prior to enrollment|desires insertion of an etonogestrel contraceptive implant for contraception|not currently pregnant or trying to become pregnant|willing to abstain from medications and supplements known to induce/inhibit cyp3a4 during the study',
 'diagnosis of type 2 diabetes, with current hba1c > 6.5% and < 12% (self-reported and then measured at screening)|a

In [9]:
# Split each Trial InclusionCriteria into list of lists of the InclusionCriteria
InclusionCriteriaList = list(map(lambda txt : txt.split("|"), InclusionCriteria))
InclusionCriteriaList = [list(filter(lambda txt: txt!= '', ltexts)) for ltexts in InclusionCriteriaList] # Delete the empty string generated by successive || (sometimes they seperate with | or ||)
print(len(InclusionCriteriaList))
InclusionCriteriaList[:2]

1000


[['adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation'],
 ['age between 18 and 40 years;',
  'women;',
  'have a regular menstrual cycle;',
  'body weight below 120 kg;',
  'bmi between 30 kg/m2 and 40 kg/m2.']]

In [10]:
### Convert the InclusionCriteria from a list of lists to a list
InclusionCriteriaList_Flat  = [item for sublist in InclusionCriteriaList for item in sublist]
print(len(InclusionCriteriaList_Flat))
InclusionCriteriaList_Flat[:5]

5572


['adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation',
 'age between 18 and 40 years;',
 'women;',
 'have a regular menstrual cycle;',
 'body weight below 120 kg;']

### Test Texts N-Grams Frequencies

In [11]:
texts = ["MK is the best of the best, MK is the hero and the savior.", 
         "The lord is our savior and MK is the best of the best",
         "Oh Lors almighty, MK too is calling himself ghe almighty",
         "Can you be our savior MK becauses you're the best of the best MK"]
ngram = 2
ngrams_frequency(texts[0], ngram)  

[('MK is', 2),
 ('is the', 2),
 ('the best', 2),
 ('best of', 1),
 ('of the', 1),
 ('best ,', 1),
 (', MK', 1),
 ('the hero', 1),
 ('hero and', 1),
 ('and the', 1),
 ('the savior', 1),
 ('savior .', 1)]

In [12]:
texts_frequencies = list(map(lambda text: ngrams_frequency(text, ngram), texts))
texts_frequencies[:2]

[[('MK is', 2),
  ('is the', 2),
  ('the best', 2),
  ('best of', 1),
  ('of the', 1),
  ('best ,', 1),
  (', MK', 1),
  ('the hero', 1),
  ('hero and', 1),
  ('and the', 1),
  ('the savior', 1),
  ('savior .', 1)],
 [('the best', 2),
  ('The lord', 1),
  ('lord is', 1),
  ('is our', 1),
  ('our savior', 1),
  ('savior and', 1),
  ('and MK', 1),
  ('MK is', 1),
  ('is the', 1),
  ('best of', 1),
  ('of the', 1)]]

In [13]:
alltexts = " ".join(texts)
alltexts

"MK is the best of the best, MK is the hero and the savior. The lord is our savior and MK is the best of the best Oh Lors almighty, MK too is calling himself ghe almighty Can you be our savior MK becauses you're the best of the best MK"

In [14]:
ngrams_frequency(alltexts, 7)[:5]

[('MK is the best of the best', 2),
 ('is the best of the best ,', 1),
 ('the best of the best , MK', 1),
 ('best of the best , MK is', 1),
 ('of the best , MK is the', 1)]

### InclusionCriteria N-Grams Frequencies

In [15]:
### Join all texts into one text
AllInclusionCriteria = " ".join(InclusionCriteriaList_Flat)
print(len(AllInclusionCriteria))
print(len(AllInclusionCriteria.split(' ')))

ngrams_frequency(AllInclusionCriteria, 1)[:5]

# => we need to remove the punctuation, stopwords, tag the words then lemmatize them.

531242
80733


[('the', 2887), (',', 2844), ('of', 2841), (')', 2205), ('to', 2140)]

In [16]:
def remove_punctuation(text):
    return "".join([i for i in text if i not in string.punctuation])

print(texts[0])
remove_punctuation(texts[0])

MK is the best of the best, MK is the hero and the savior.


'MK is the best of the best MK is the hero and the savior'

In [17]:
def remove_stopwords(text):
    return ' '.join([word for word in word_tokenize(text) if not word in stopwords.words('english')])

print(texts[0])
remove_stopwords(texts[0])

MK is the best of the best, MK is the hero and the savior.


'MK best best , MK hero savior .'

'''
Part-of-speech tagging, words, or tokens are assigned part of speech tags, which are typically morphosyntactic subtypes of fundamental syntactic 
categories in the language such as a noun, or verb. By lemmatizing lexemes, inflected forms of a word are grouped together under a common root. 

The tagging and lemmatization of parts of speech are essential to linguistic pre-processing. This website uses morphosyntactic descriptors and 
part-of-speech tagging as acronyms. In the context of the NLTK Lemmatization, the part of speech tags are pre-defined with shortcuts for the 
NLTK WordNetLemmatizer as below.

https://www.holisticseo.digital/python-seo/nltk/lemmatize
'''

In [18]:
# Custom tagger, so the tags can be interpreted by the WordNetLemmatizer()
# Because NLTK nltk.pos_tag() and WordNetLemmatizer() do not use the same naming covention
def nltk_pos_tagger(nltk_tag):
  
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [46]:
def tag_lemmatize_sentence(sentence):

    # Tag the Tokenized text
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  

    # Aply the custom tagger, so the tags can be interpreted by the WordNetLemmatizer()
    wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), nltk_tagged)
    
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:        
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

sentence = "I am voting for better that politician. in this NLTK, Lemmatization example sentences"
print(sentence) 
print(tag_lemmatize_sentence(sentence))

I am voting for better that politician. in this NLTK, Lemmatization example sentences
I be vote for good that politician . in this NLTK , Lemmatization example sentence


In [47]:
print(sentence)
print(remove_punctuation(sentence))
print(remove_stopwords(remove_punctuation(sentence)))
print(lemmatizer.lemmatize(remove_stopwords(remove_punctuation(sentence))))
print(tag_lemmatize_sentence(remove_stopwords(remove_punctuation(sentence))))

I am voting for better that politician. in this NLTK, Lemmatization example sentences
I am voting for better that politician in this NLTK Lemmatization example sentences
I voting better politician NLTK Lemmatization example sentences
I voting better politician NLTK Lemmatization example sentences
I vote well politician NLTK Lemmatization example sentence


In [50]:
### Join all texts into one text
AllInclusionCriteria = " ".join(InclusionCriteriaList_Flat)
print(len(AllInclusionCriteria))
print(len(AllInclusionCriteria.split(' ')))

ngrams_frequency(AllInclusionCriteria, 1)[:5]
# => we need to remove the punctuation, stopwords, tag the words then lemmatize them.

531242
80733


[('the', 2887), (',', 2844), ('of', 2841), (')', 2205), ('to', 2140)]

In [51]:
%%time
ProcessedAllInclusionCriteria = tag_lemmatize_sentence(remove_stopwords(remove_punctuation(AllInclusionCriteria)))

CPU times: user 14 s, sys: 1.22 s, total: 15.3 s
Wall time: 15.3 s


In [53]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 1)[:5]

CPU times: user 564 ms, sys: 7.78 ms, total: 571 ms
Wall time: 944 ms


[('patient', 816),
 ('year', 733),
 ('study', 727),
 ('age', 642),
 ('consent', 538)]

In [54]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 2)[:5]

CPU times: user 570 ms, sys: 10.9 ms, total: 581 ms
Wall time: 601 ms


[('informed consent', 286),
 ('18 year', 244),
 ('year old', 189),
 ('year age', 156),
 ('age 18', 141)]

In [55]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 3)[:5]

CPU times: user 585 ms, sys: 7.4 ms, total: 592 ms
Wall time: 649 ms


[('write informed consent', 133),
 ('sign informed consent', 80),
 ('18 year age', 75),
 ('18 year old', 73),
 ('age 18 year', 62)]

In [56]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 4)[:5]

CPU times: user 572 ms, sys: 11.2 ms, total: 583 ms
Wall time: 621 ms


[('provide write informed consent', 47),
 ('age ≥ 18 year', 42),
 ('eastern cooperative oncology group', 34),
 ('upper limit normal uln', 28),
 ('group ecog performance status', 27)]

In [57]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 5)[:5]

CPU times: user 601 ms, sys: 10.8 ms, total: 612 ms
Wall time: 684 ms


[('eastern cooperative oncology group ecog', 26),
 ('cooperative oncology group ecog performance', 24),
 ('oncology group ecog performance status', 24),
 ('x upper limit normal uln', 16),
 ('able provide write informed consent', 15)]

In [58]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 6)[:5]

CPU times: user 597 ms, sys: 14.6 ms, total: 611 ms
Wall time: 751 ms


[('eastern cooperative oncology group ecog performance', 24),
 ('cooperative oncology group ecog performance status', 24),
 ('15 x upper limit normal uln', 11),
 ('response evaluation criterion solid tumor recist', 10),
 ('within 14 day prior study registration', 9)]

In [59]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 7)[:5]

CPU times: user 599 ms, sys: 8.3 ms, total: 608 ms
Wall time: 680 ms


[('eastern cooperative oncology group ecog performance status', 24),
 ('cooperative oncology group ecog performance status 0', 8),
 ('ability understand willingness sign write informed consent', 7),
 ('oncology group ecog performance status 0 1', 7),
 ('bilirubin 15 x upper limit normal uln', 7)]

In [60]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 8)[:5]

CPU times: user 610 ms, sys: 5.79 ms, total: 616 ms
Wall time: 687 ms


[('eastern cooperative oncology group ecog performance status 0', 8),
 ('cooperative oncology group ecog performance status 0 1', 7),
 ('perform within 42 day sign screen leukapheresis consent', 6),
 ('state willingness comply study procedure availability duration study', 5),
 ('total bilirubin 15 x upper limit normal uln', 5)]

In [61]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 9)[:5]

CPU times: user 484 ms, sys: 7.37 ms, total: 492 ms
Wall time: 542 ms


[('eastern cooperative oncology group ecog performance status 0 1', 7),
 ('result failure rate 1 per year treatment period least', 4),
 ('eastern cooperative oncology group ecog performance status ≤ 2', 4),
 ('consent obtain studyrelated activity studyrelated activity procedure relate recording',
  4),
 ('obtain studyrelated activity studyrelated activity procedure relate recording data',
  4)]

In [62]:
%%time
ngrams_frequency(ProcessedAllInclusionCriteria, 10)[:5]

CPU times: user 607 ms, sys: 7.11 ms, total: 614 ms
Wall time: 664 ms


[('consent obtain studyrelated activity studyrelated activity procedure relate recording data',
  4),
 ('obtain studyrelated activity studyrelated activity procedure relate recording data accord',
  4),
 ('studyrelated activity studyrelated activity procedure relate recording data accord protocol',
  4),
 ('male female age equal 18 year time signing informed consent', 4),
 ('study subject inform study purpose protocal benefit risk sign informed', 4)]